# Service Restoration Observatory

# Changes occuring in the provision of pulse oximetry related to COVID-19: 07-2020 - 02-21

Below are various time-series graphs showing changes in total nationwide the provision of pulse oximetry related to COVID-19 as well as a breakdown by region and other demographic variables.  The following graphs are available:

* [Total pulse oximetry number](#total)
    * [Total pulse oximetry number by child codes](#child-codes)
* [Total pulse oximetry number by practice](#practice)
* [Breakdown by region](#region)
* [Breakdown by age band](#age)
* [Breakdown by sex](#sex)


#### Methods
Using OpenSAFELY-TPP, covering 40% of England's population, we have assessed coding activity related to pulse oximetry related to COVID-19 in general practice from the beginning of July 2020 until the end of January 2021. The codelist used can be found here at [OpenSAFELY Codelists](https://codelists.opensafely.org/codelist/opensafely/pulse-oximetry/72ce1380/).  For each month within the study period, we have calculated the rate at which the code was recorded per 1000 registered patients.

All analytical code and output is available for inspection at the [OpenSAFELY GitHub repository](https://github.com/opensafely/SRO-pulse-oximetry).

In [ ]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from IPython.display import HTML
from IPython.core.display import HTML as Center
from utilities import *

%matplotlib inline

Center(""" <style>
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
}
</style> """)

In [ ]:
# Load measures df
measures_df_total = pd.read_csv('../output/measures/measure_had_pulse_ox_total.csv')
measures_df_event_code = pd.read_csv('../output/measures/measure_had_pulse_ox_event_code.csv')
measures_df_practice = pd.read_csv('../output/measures/measure_had_pulse_ox_practice_only.csv')
measures_df_by_region = pd.read_csv('../output/measures/measure_had_pulse_ox_by_region.csv')
measures_df_by_age = pd.read_csv('../output/measures/measure_had_pulse_ox_by_age_band.csv')
measures_df_by_sex = pd.read_csv('../output/measures/measure_had_pulse_ox_by_sex.csv')

codelist = pd.read_csv('../codelists/opensafely-pulse-oximetry.csv')
codelist.round(16)


pulse_ox_code_mapping = {"1325251000000106": "Y2a44",
                         "1325261000000109": "Y2a45",
                         "1325271000000102": "Y2a46",
                         "1325201000000105": "Y2a47",
                         "1325191000000108": "Y2a48", 
                         "1325221000000101": "Y2a49",
                         "1325241000000108": "Y2a4a",
                         "1325281000000100": "Y2a4b",
                         "1325681000000102": "Y2b97",
                         "1325701000000100": "Y2b98",
                         "1325691000000100": "Y2b99",
                         "1325211000000107": "YA796"
                         }
                         

def apply_code_mapping(row):
    row['CTV3ID'] = pulse_ox_code_mapping[str(row['code'])]
    return row

codelist = codelist.apply(lambda row: apply_code_mapping(row), axis=1)


measures_df_event_code.round(16)

# temporary fix for population not working in Measures
measures_df_total = measures_df_total.groupby(
    ['date'])['had_pulse_ox', 'population'].sum().reset_index()
measures_df_total['value'] = measures_df_total['had_pulse_ox'] /measures_df_total['population']



# Get measures

class Measure:
  def __init__(self, id, numerator, denominator, group_by):
    self.id = id
    self.numerator = numerator
    self.denominator = denominator
    self.group_by = group_by
    
    
measures = [
   

    Measure(
        id="had_pulse_ox_total",
        numerator="had_pulse_ox",
        denominator="population",
        group_by=None
    ),

    Measure(
        id="had_pulse_ox_event_code",
        numerator="had_pulse_ox",
        denominator="population",
        group_by=["had_pulse_ox_event_code"]
    ),

    Measure(
        id="had_pulse_ox_practice_only",
        numerator="had_pulse_ox",
        denominator="population",
        group_by=["practice"]
    ),

    Measure(
        id="had_pulse_ox_by_region",
        numerator="had_pulse_ox",
        denominator="population",
        group_by=["region"],
    ),

    Measure(
        id="had_pulse_ox_by_sex",
        numerator="had_pulse_ox",
        denominator="population",
        group_by=["sex"],
    ),

    Measure(
        id="had_pulse_ox_by_age_band",
        numerator="had_pulse_ox",
        denominator="population",
        group_by=["age_band"],
    ),


    
]


# Convert date column to datetime and sort by date
to_datetime_sort(measures_df_total)
to_datetime_sort(measures_df_event_code)
to_datetime_sort(measures_df_practice)
to_datetime_sort(measures_df_by_region)
to_datetime_sort(measures_df_by_age)
to_datetime_sort(measures_df_by_sex)


# Redact small numbers from measures df
# redact_small_numbers(measures_df_total, 5, measures[0])

# Calculate rates

calculate_rate(measures_df_total, value_col='had_pulse_ox', population_col='population', rate_per=1000)
calculate_rate(measures_df_event_code, value_col='had_pulse_ox', population_col='population', rate_per=1000)
calculate_rate(measures_df_practice, value_col='had_pulse_ox', population_col='population', rate_per=1000)
calculate_rate(measures_df_by_region, value_col='had_pulse_ox', population_col='population', rate_per=1000)
calculate_rate(measures_df_by_age, value_col='had_pulse_ox', population_col='population', rate_per=1000)
calculate_rate(measures_df_by_sex, value_col='had_pulse_ox', population_col='population', rate_per=1000)


<a id='total'></a>
## Total Pulse Oximetry Number

In [ ]:
plot_measures(measures_df_total, title='Total pulse oximetry across whole population', measure_id='measure_1', column_to_plot='rate', category=False, y_label='Rate per 1000')

<a id='child-codes'></a>
### Sub totals by child codes

Events for the top pulse oximetry subcodes across the study period

In [ ]:
child_table = create_child_table(df=measures_df_event_code, code_df=codelist, code_column='CTV3ID', term_column='term', measure='had_pulse_ox')
child_table

In [ ]:
def plot_measures(df, title, measure_id, column_to_plot, category=False, y_label='Rate per 1000', interactive=True):

    if interactive:

        fig = go.Figure()

        if category:
            for unique_category in df[category].unique():

                df_subset = df[df[category] == unique_category]
                fig.add_trace(go.Scatter(
                    x=df_subset['date'], y=df_subset[column_to_plot], name=unique_category))

        else:
            fig.add_trace(go.Scatter(
                x=df['date'], y=df[column_to_plot]))

        # Set title
        fig.update_layout(
            title_text=title,
            hovermode='x',
            title_x=0.5,


        )

        fig.update_yaxes(title=y_label)
        fig.update_xaxes(title="Date")

        # Add range slider
        fig.update_layout(
            xaxis=go.layout.XAxis(
                rangeselector=dict(
                    buttons=list([
                        dict(count=1,
                            label="1m",
                            step="month",
                            stepmode="backward"),
                        dict(count=6,
                            label="6m",
                            step="month",
                            stepmode="backward"),

                        dict(count=1,
                            label="1y",
                            step="year",
                            stepmode="backward"),
                        dict(step="all")
                    ])
                ),
                rangeslider=dict(
                    visible=True
                ),
                type="date"
            )
        )

        fig.show()

    else:

        if category:
            for unique_category in df[category].unique():

                df_subset = df[df[category] == unique_category]

                plt.plot(df_subset['date'], df_subset[column_to_plot], marker='o')
        else:
            plt.plot(df['date'], df[column_to_plot], marker='o')

        plt.ylabel(y_label)
        plt.xlabel('Date')
        plt.xticks(rotation='vertical')
        plt.title(title)

        if category:
            plt.legend(df[category].unique(), bbox_to_anchor=(
                1.04, 1), loc="upper left")

        else:
            pass

        plt.savefig(f'output/{measure_id}.jpeg', bbox_inches='tight')
        plt.show()
        plt.clf()



In [ ]:
measures_df_event_code['had_pulse_ox_event_code'] = measures_df_event_code['had_pulse_ox_event_code'].astype(str)

In [ ]:
plot_measures(measures_df_event_code, title='Total pulse oximetry across whole populatio for top 5 child codes', measure_id='measure_had_pulse_ox', column_to_plot='rate', category='had_pulse_ox_event_code', y_label='Rate per 1000')

<a id='practice'></a>
## Total number by practice

In [ ]:
practice_df = pd.read_csv('../output/input_practice_count.csv')
practices_dict =calculate_statistics_practices(measures_df_practice, practice_df,"2020-01-01")
print(f'Practices included entire period: {practices_dict["total"]["number"]} ({practices_dict["total"]["percent"]}%)')
print(f'Practices included within last year: {practices_dict["year"]["number"]} ({practices_dict["year"]["percent"]}%)')
print(f'Practices included within last 3 months: {practices_dict["months_3"]["number"]} ({practices_dict["months_3"]["percent"]}%)')



In [ ]:
interactive_deciles_chart(measures_df_practice, period_column='date', column='had_pulse_ox', title='Decile chart',ylabel='rate per 1000')

<a id="region"></a>
## Breakdown by Region

Number of events within each group.

In [ ]:
measures_df_by_region['region'] = measures_df_by_region['region'].replace(np.nan, 'NA')
counts_df = calculate_statistics_demographics(df=measures_df_by_region, demographic_var='region', end_date="2021-01-01", event_column='had_pulse_ox')
counts_df

In [ ]:
plot_measures(measures_df_by_region, title='Breakdown by region',measure_id='measure_had_pulse_ox', column_to_plot='rate', category='region', y_label='Rate per 1000')


<a id="age"></a>
## Breakdown by Age

Number of events within each group.

In [ ]:
counts_df = calculate_statistics_demographics(df=measures_df_by_age, demographic_var='age_band', end_date="2021-01-01", event_column='had_pulse_ox')
counts_df

In [ ]:
plot_measures(measures_df_by_age, title='Breakdown by age',measure_id='measure_had_pulse_ox', column_to_plot='rate', category='age_band', y_label='Rate per 1000')


<a id="sex"></a>
## Breakdown by Sex

Number of events within each group.

In [ ]:
counts_df = calculate_statistics_demographics(df=measures_df_by_sex, demographic_var='sex', end_date="2021-02-01", event_column='had_pulse_ox')
counts_df

In [ ]:
plot_measures(measures_df_by_sex, title='Breakdown by sex',measure_id='measure_1', column_to_plot='rate', category='sex', y_label='Rate per 1000')